In [1]:
# Load environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
from langchain_community.llms import Ollama
from langchain_openai import OpenAI

from langchain.tools import tool
from langchain import hub
from langchain.prompts import load_prompt

from langchain.agents import AgentExecutor, create_react_agent

In [3]:
# Switch to use openAI
#llm = OpenAI(temperature=0, verbose=True, model="gpt-3.5-turbo-instruct") # defautl model : "gpt-3.5-turbo-instruct"
llm = Ollama(model="mistral")

In [4]:
# custom simple tool
@tool
def square(x: int) -> int:
    """
    squares a number
    """
    return int(x)**2

In [5]:
# tool features
print(square.name)
print(square.description)
print(square.args)

square
square(x: int) -> int - squares a number
{'x': {'title': 'X', 'type': 'integer'}}


In [6]:
# pull prompt from the hub and saves it 
obj = hub.pull("hwchase17/react")
obj.save("prompt.json")

In [7]:
# inititalize prompt and tools for custom agent
prompt = load_prompt('prompt.json')
tools = [
    square
    ]

In [16]:
# the prompt template
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [8]:
agent_custom = create_react_agent(
     llm,
     tools,
    prompt
    )

agent_executor = AgentExecutor(agent=agent_custom, tools=tools, verbose=True)

In [9]:
agent_executor.invoke(input= {'input':"square of 5"})



> Entering new AgentExecutor chain...
 I need to find the square of a given number. The number is 5.
Action: square
Action Input: 525 I now know the final answer.
Final Answer: The square of 5 is 25.

> Finished chain.


{'input': 'square of 5', 'output': 'The square of 5 is 25.'}